In [1]:
import pandas as pd
import numpy as np

import os

## Summary

This notebook is designed to apply necessary transformations to the original data in order to use it for machine learning purposes. The following steps will be applied:

- Clean up store duplicates
- Drop item sale with negative price
- Aggregate item sales on a monthly basis

In [2]:
train_df = pd.read_csv(os.path.join("..", "data", "sales_train.csv"))
shop_df = pd.read_csv(os.path.join("..", "data", "shops.csv"))

mod_train_df = train_df.copy()

### Drop store duplicates

In [3]:
shop_df.iloc[[0,57,1,58,10,11]]

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
57,"Якутск Орджоникидзе, 56",57
1,"!Якутск ТЦ ""Центральный"" фран",1
58,"Якутск ТЦ ""Центральный""",58
10,Жуковский ул. Чкалова 39м?,10
11,Жуковский ул. Чкалова 39м²,11


In [4]:
mod_train_df.loc[mod_train_df.shop_id == 0, "shop_id"] = 57
mod_train_df.loc[mod_train_df.shop_id == 1, "shop_id"] = 58
mod_train_df.loc[mod_train_df.shop_id == 11, "shop_id"] = 10

I checked the test set and stores 0, 1 and 11 are not even present in there.

## Drop negative price items

In [5]:
mod_train_df = mod_train_df.drop(mod_train_df[mod_train_df.item_price < 0].index)

## Aggregate monthly

In [6]:
mod_train_df = mod_train_df.drop("date", axis=1)

mod_train_df = (
    mod_train_df
    .groupby(["shop_id", "item_id", "date_block_num"])
    .agg({
        "item_price": np.mean,
        "item_cnt_day": np.sum,
    })
    .sort_values(["date_block_num", "item_id"])
    .reset_index()
)

In [7]:
mod_train_df.head()

,shop_id,item_id,date_block_num,item_price,item_cnt_day
0,25,19,0,28.0,1.0
1,2,27,0,2499.0,1.0
2,10,27,0,1890.0,1.0
3,19,27,0,2499.0,1.0
4,26,27,0,2499.0,1.0


In [8]:
mod_train_df.to_csv(os.path.join("..", "data", "mod_sales_train.csv"), index=False)